# **Predicting weather in London with MLflow**

> *Perform a machine learning experiment to find the best model that predicts the temperature in London!*






As the climate changes, predicting the weather becomes ever more important for businesses. The task is  to support on a machine learning project with the aim of building a pipeline to predict the climate in London, England. Specifically, the model should predict mean temperature in degrees Celsius (°C).

Since the weather depends on a lot of different factors, we will want to run a lot of experiments to determine what the best approach is to predict the weather. In this project, we will run experiments for different regression models predicting the mean temperature, using a combination of sklearn and mlflow.

We will be working with data stored in london_weather.csv, which contains the following columns:

- date - recorded date of measurement - (int)
- cloud_cover - cloud cover measurement in oktas - (float)
- sunshine - sunshine measurement in hours (hrs) - (float)
- global_radiation - irradiance measurement in Watt per square meter (W/m2) - (float)
- max_temp - maximum temperature recorded in degrees Celsius (°C) - (float)
- mean_temp - target mean temperature in degrees Celsius (°C) - (float)
- min_temp - minimum temperature recorded in degrees Celsius (°C) - (float)
- precipitation - precipitation measurement in millimeters (mm) - (float)
- pressure - pressure measurement in Pascals (Pa) - (float)
- snow_depth - snow depth measurement in centimeters (cm) - (float)





Dataset link :  https://www.kaggle.com/datasets/emmanuelfwerr/london-weather-data

Steps are-

- Use machine learning to predict the mean temperature in London, England, logging your root mean squared error (RMSE) metrics using mlflow.

- Build a model to predict "mean_temp" with a RMSE of 3 or less.
- Use MLflow to log any models you train, their hyperparameters, and respective RMSE scores (ensuring you include "rmse" as part of the metric name).
- Search all of your mlflow runs and store the results as a variable called experiment_results


In [1]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 555.1/555.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
# Run this cell to import the modules you require
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Read in the data
weather = pd.read_csv("london_weather.csv")


In [5]:
# Imputer for features
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

# Define features and target
features = ['cloud_cover', 'sunshine', 'global_radiation', 'max_temp',
            'min_temp', 'precipitation', 'pressure', 'snow_depth']
X = weather[features]

# Impute missing values in features
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

In [6]:
# Handle missing values in target column (mean_temp)
y_imputer = SimpleImputer(strategy='mean')
y = y_imputer.fit_transform(weather['mean_temp'].values.reshape(-1, 1)).ravel()  # Convert to NumPy, reshape, and impute

# Now proceed with splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
# 2. Function to train and log models using mlflow
def train_and_log_model(model, model_name, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        # Log model, hyperparameters, and metrics to MLflow
        mlflow.sklearn.log_model(model, model_name)
        mlflow.log_metric("rmse", rmse)

        print(f"{model_name}: RMSE = {rmse}")
        return rmse

# Initialize models
linear_model = LinearRegression()
tree_model = DecisionTreeRegressor(random_state=42)
forest_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train models and log results
train_and_log_model(linear_model, "LinearRegression", X_train, X_test, y_train, y_test)
train_and_log_model(tree_model, "DecisionTreeRegressor", X_train, X_test, y_train, y_test)
train_and_log_model(forest_model, "RandomForestRegressor", X_train, X_test, y_train, y_test)

2024/09/21 20:12:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LinearRegression: RMSE = 0.9166133728599348


2024/09/21 20:12:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


DecisionTreeRegressor: RMSE = 1.2750102381614619


2024/09/21 20:12:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForestRegressor: RMSE = 0.9166052782128887


0.9166052782128887

In [7]:
# Search for MLflow runs and store them
experiment_results = mlflow.search_runs(order_by=["metrics.rmse ASC"])
experiment_results

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name
0,8c53d9bf04e14f0c9895a859b8592f31,0,FINISHED,file:///content/mlruns/0/8c53d9bf04e14f0c9895a...,2024-09-21 20:12:29.001000+00:00,2024-09-21 20:12:39.863000+00:00,0.916605,"[{""run_id"": ""8c53d9bf04e14f0c9895a859b8592f31""...",mercurial-shrew-914,root,LOCAL,/usr/local/lib/python3.10/dist-packages/colab_...
1,af511605ebcf40f984f3615c1ccbb836,0,FINISHED,file:///content/mlruns/0/af511605ebcf40f984f36...,2024-09-21 20:12:21.399000+00:00,2024-09-21 20:12:26.455000+00:00,0.916613,"[{""run_id"": ""af511605ebcf40f984f3615c1ccbb836""...",dazzling-ox-698,root,LOCAL,/usr/local/lib/python3.10/dist-packages/colab_...
2,86227f22bfbb401dbe1887897487459c,0,FINISHED,file:///content/mlruns/0/86227f22bfbb401dbe188...,2024-09-21 20:12:26.460000+00:00,2024-09-21 20:12:28.997000+00:00,1.275010,"[{""run_id"": ""86227f22bfbb401dbe1887897487459c""...",loud-gull-256,root,LOCAL,/usr/local/lib/python3.10/dist-packages/colab_...
